# This is the new version of Skimpy from github

In [6]:
from skimpy.io.yaml import load_yaml_model, export_to_yaml

met_model = 'ecoli_shikki_regulation'
base = f'.'
path_to_kmodel = f'{base}/kinetic_model_scaffold.yaml'
output_model_path = f'{base}/output/kin_varma_curated_regulation.yaml'
kmodel = load_yaml_model(path_to_kmodel)

2024-03-01 18:14:50,684 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


These parameters are omitted due to KeyError: 
['hill_coefficient_ATPM', 'k1_bwd_ACALDtex', 'k1_bwd_ACtex', 'k1_bwd_AKGtex', 'k1_bwd_CO2tex', 'k1_bwd_D-LACtex', 'k1_bwd_ETOHtex', 'k1_bwd_FORtex', 'k1_bwd_GLCtex', 'k1_bwd_GLUtex', 'k1_bwd_Htex', 'k1_bwd_MALtex', 'k1_bwd_NH4tex', 'k1_bwd_O2tex', 'k1_bwd_PHEtex', 'k1_bwd_PYRtex', 'k1_bwd_SERtex', 'k1_bwd_SUCCtex', 'k1_bwd_TRPtex', 'k1_bwd_TYRtex', 'k1_fwd_ACALDtex', 'k1_fwd_ACtex', 'k1_fwd_AKGtex', 'k1_fwd_CO2tex', 'k1_fwd_D-LACtex', 'k1_fwd_ETOHtex', 'k1_fwd_FORtex', 'k1_fwd_GLCtex', 'k1_fwd_GLUtex', 'k1_fwd_Htex', 'k1_fwd_MALtex', 'k1_fwd_NH4tex', 'k1_fwd_O2tex', 'k1_fwd_PHEtex', 'k1_fwd_PYRtex', 'k1_fwd_SERtex', 'k1_fwd_SUCCtex', 'k1_fwd_TRPtex', 'k1_fwd_TYRtex', 'k2_bwd_ACALDtex', 'k2_bwd_ACtex', 'k2_bwd_AKGtex', 'k2_bwd_CO2tex', 'k2_bwd_D-LACtex', 'k2_bwd_ETOHtex', 'k2_bwd_FORtex', 'k2_bwd_GLCtex', 'k2_bwd_GLUtex', 'k2_bwd_Htex', 'k2_bwd_MALtex', 'k2_bwd_NH4tex', 'k2_bwd_O2tex', 'k2_bwd_PHEtex', 'k2_bwd_PYRtex', 'k2_bwd_SERtex', 'k2_

In [7]:
from skimpy.io.regulation import load_enzyme_regulation
import pandas as pd
# Add regulation data to kinetic model
path_to_regulation_data = f'{base}/allosteric_regulations.csv'
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel.reactants.keys()))]

kmodel = load_enzyme_regulation(kmodel, df_regulations_all)

TabDict([('DSM_h_c', <skimpy.core.modifiers.DisplacementSmallMoleculeModifier object at 0x7f2e43185da0>), ('DSM_h_p', <skimpy.core.modifiers.DisplacementSmallMoleculeModifier object at 0x7f2e43185278>)])
Added regulation reaction ACALD of with <skimpy.mechanisms.convenience_with_inihibition.make_convenience_with_inhibition.<locals>.ConvenienceInhibited object at 0x7f2e431b6978> kinetics 
Added regulation reaction ADK1 of with <skimpy.mechanisms.convenience_with_inihibition.make_convenience_with_inhibition.<locals>.ConvenienceInhibited object at 0x7f2e431b6a90> kinetics 
Added regulation reaction ANS of with <skimpy.mechanisms.convenience_with_inihibition.make_convenience_with_inhibition.<locals>.ConvenienceInhibited object at 0x7f2e431b6ef0> kinetics 
Added regulation reaction CHORM of with <skimpy.mechanisms.generalized_reversible_hill_n_n_h1_with_inhibition.make_generalized_reversible_hill_n_n_h1_with_inhibition.<locals>.H1GeneralizedReversibleHillInhibited object at 0x7f2e431b67b8> 

In [14]:
no_km = 0
no_ka = 0
no_ki = 0
k_list = []
for k,v in kmodel.parameters.items():
    if k.startswith("km_"):
        no_km += 1 
        k_list.append(k)
    elif ("activation" in k):
        no_ka += 1
        k_list.append(k)
    elif ("inhibition" in k):
        no_ki += 1
        k_list.append(k)
print(f"The model has {len(kmodel.reactions)} reactions.")
print(f"The number of km in this model is {no_km}.")
print(f"The number of inhibition k in this model is {no_ki}.")
print(f"The number of activation k in this model is {no_ka}.")


The model has 123 reactions.
The number of km in this model is 384.
The number of inhibition k in this model is 12.
The number of activation k in this model is 10.


In [16]:
import pickle

file_name = "./output/parameter_names_k_with_regulation.pkl"
open_file = open(file_name, "wb")
pickle.dump(k_list, open_file)
open_file.close()

In [10]:
export_to_yaml(kmodel, output_model_path)

True